In [3]:
#more advanced version of AGM-adabelief which evaluates the signal for each component separately
import numpy as np
import pdb

import fire
import matplotlib.pyplot as plt
import torch
import torch.optim.swa_utils as swa_utils
import torchcde
import torchsde
from torch import nn
from torchdiffeq import odeint_adjoint as odeint
import copy
import matplotlib.pyplot as plt
from torch import profiler
from tqdm import tqdm
import numpy as np
from typing import Sequence
import pdb
from torchdiffeq import odeint
from helpers import KuramotoAGM, objective, adj_lattice, criticalK,adjoint_init, KuramotoAdjoint,  order_param,  adabelief_update, grad_estimate_vec, adabelief_update_manifold, combine_vector_field, AGM_ODE, ODE_Adjoint
import psutil
import os
import networkx as nx
from IPython.display import clear_output
is_cuda = torch.cuda.is_available()
device = 'cuda' if is_cuda else 'cpu'
#device = 'cpu'
if not is_cuda:
    print("Warning: CUDA not available; falling back to CPU but this is likely to be very slow.")
torch.cuda.empty_cache()
import gc
import pickle

In [5]:


batch_size = 1024
scale=5
scale_kin=5
#read adjacency matrix and frequencies
%store -r Astored
state_size=Astored.shape[1]

%store -r freqs
freqss=freqs.cpu().detach().numpy()

#calculate critical value
Kcrit, L=criticalK(Astored,state_size, scale)
Kcrit = 0.01 * Kcrit


A=torch.tensor(Astored).to(torch.float32).to(device)
L=torch.tensor(L).to(torch.float32).to(device)


print(Kcrit)
print(Astored.shape)


0.841310456374586
(64, 64)


/home/qtomcat/anaconda3/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [21]:
t_total=1
t_size=32
dt=torch.tensor(t_total/t_size).to(device)

#initialize components of vector field
temporal=torch.zeros(t_size,batch_size,1).to(device)
components= torch.ones(1, state_size).to(device)
components= components/ torch.sqrt((torch.sum(components**2, dim=1)).unsqueeze(1))
components = components.repeat(1,batch_size,1)

force_field= temporal*components

#forward solver
SML= AGM_ODE(batch_size, state_size, Kcrit,A, freqs, force_field, dt).to(device)

#variables for optimizer    
m = torch.zeros_like(components).to(device)
v = torch.zeros_like(components).to(device)
v_hat_max = torch.zeros_like(components).to(device)

mt = torch.zeros_like(temporal).to(device)
vt = torch.zeros_like(temporal).to(device)
vt_hat_max = torch.zeros_like(temporal).to(device)


adam_steps=0
adam_steps_t=0

#weight
R=0.0001
#total iterations
iter = 6000

perf=[]
st_perf=[]
orpm=[]


t = np.linspace(0, iter, iter)


#store -r ys_ran
ys_ran =torch.randn(batch_size, state_size).to(device)/scale_kin


In [19]:

sched1=1
sched2=1
alpha=1
ema_loss=0
theta=0.005
#backward solver
SMD= ODE_Adjoint(batch_size, state_size, Kcrit, freqs,A, force_field, ys_ran, dt, device).to(device)

#init values for the state
ys_ini = torch.zeros(batch_size, 1).to(device)
ts_exp = torch.linspace(0, t_total, t_size).to(device)
ys_exp = torch.cat((ys_ran,ys_ini),dim=1).to(device)

#start learning
for j in tqdm(range(iter)):

    print(j)
    force_field= temporal*components
    #solve forward
    with torch.no_grad():
        ys_tray = odeint(SML, ys_exp, ts_exp, method = 'heun2')

    #calculate loss
    obj= objective(ys_tray[-1,:,0:state_size],A)
    kin= R*ys_tray[-1,:,-1]
    loss=obj+kin 
  
    
    print("loss")
    print(loss.mean())
    #the value of adjoint at t=T
    ys=ys_tray[:,:,0:state_size]
    initials=adjoint_init(ys_tray[-1,:,0:state_size],state_size)

    SMD.x=ys
    SMD.u=force_field
    
   
    with torch.no_grad():
        p_tray = odeint(SMD, initials, ts_exp, method = 'heun2')

    orparam=order_param(ys_tray[-1,:,0:state_size],L)
    print("order param")
    print(torch.mean(orparam))
    #compute gradients
    p=torch.flip(p_tray, [0])
    
    comp_grad, scal_grad=grad_estimate_vec(p,ys,temporal,components,state_size,R,Kcrit)
    beta_grad_comp=  Kcrit*torch.sum(scal_grad, dim=0)
    beta_grad_scal=  2*R*temporal +(Kcrit* torch.sum(comp_grad,dim=2)).unsqueeze(2)
    #update the components of vector field with optimizer
    with torch.no_grad():
        
        temporal, mt, vt, vt_hat_max, adam_steps_t = adabelief_update(temporal, beta_grad_scal, mt, vt, vt_hat_max, adam_steps_t, lr=theta*(sched1**adam_steps_t ))
    
        components, m, v, v_hat_max, adam_steps = adabelief_update_manifold(components, beta_grad_comp.unsqueeze(0), m, v, v_hat_max, adam_steps, lr=theta*(sched2**adam_steps))
 
 
    force_field= temporal*components
    #save data
    SML.u=force_field
    perf.append(loss.mean().cpu().detach().numpy())
    st_perf.append((loss.std()/np.sqrt(batch_size)).cpu().detach().numpy())
    orpm.append((torch.mean(orparam)).cpu().detach().numpy())

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()
    #if j%1000==0:
        #clear_output(wait=False)
plt.plot(t,np.array(perf))

plt.fill_between(t, np.array(perf)-np.array(erpe), np.array(perf)+np.array(erpe))
plt.show()

  0%|          | 1/6000 [00:00<14:40,  6.81it/s]

0
loss
tensor(81.7218, device='cuda:0')
order param
tensor(0.9776, device='cuda:0')
1
loss
tensor(81.7032, device='cuda:0')


  0%|          | 2/6000 [00:00<14:23,  6.95it/s]

order param
tensor(0.9776, device='cuda:0')
2
loss
tensor(81.6826, device='cuda:0')
order param
tensor(0.9777, device='cuda:0')


  0%|          | 4/6000 [00:00<14:17,  6.99it/s]

...

  0%|          | 20/6000 [00:02<13:35,  7.33it/s]

19
loss
tensor(80.9055, device='cuda:0')
order param
tensor(0.9779, device='cuda:0')
20
loss
tensor(80.8441, device='cuda:0')
order param
tensor(0.9779, device='cuda:0')


  0%|          | 21/6000 [00:02<14:03,  7.08it/s]


21
loss
tensor(80.7818, device='cuda:0')



KeyboardInterrupt

